In [9]:
!pip install torch torchvision numpy pillow
!git clone https://github.com/IrvingMeng/MagFace.git
!pip install -r /content/MagFace/raw/requirements.txt

import math
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
from torchvision import models

fatal: destination path 'MagFace' already exists and is not an empty directory.
  Using cached torchshard-0.1.0-py3-none-any.whl.metadata (748 bytes)
  Using cached jupyterlab-4.3.1-py3-none-any.whl.metadata (16 kB)
  Using cached Pillow-6.2.2.tar.gz (37.8 MB)
  Preparing metadata (setup.py) ... done
  Using cached loguru-0.7.2-py3-none-any.whl.metadata (23 kB)
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [14]:
class MagFace(nn.Module):
    def __init__(self):
        super(MagFace, self).__init__()
        self.model = models.resnet50()
        self.model.fc = nn.Identity()

    def forward(self, x):
        return self.model(x)

from google.colab import drive
drive.mount('/content/drive')

model = MagFace()
checkpoint = torch.load("/content/drive/My Drive/faceRecognition/magface_iresnet50_MS1MV2_ddp_fp32.pth", map_location=torch.device('cpu'))
model.load_state_dict(checkpoint, strict=False)
model.eval()

def preprocess_image(image_path):
    preprocess = transforms.Compose([
        transforms.Resize((112, 112)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    img = Image.open(image_path).convert("RGB")
    img_tensor = preprocess(img).unsqueeze(0)
    return img_tensor

def get_embedding(model, image_path):
    image_tensor = preprocess_image(image_path)
    with torch.no_grad():
        embedding = model(image_tensor)
    return embedding.squeeze(0).numpy()

image1_path = "/content/drive/My Drive/faceRecognition/1-2.jpg"
image2_path = "/content/drive/My Drive/faceRecognition/test1.jpg"

embedding1 = get_embedding(model, image1_path)
embedding2 = get_embedding(model, image2_path)

def distance_(embeddings0, embeddings1):
    # Distance based on cosine similarity
    dot = np.sum(np.multiply(embeddings0, embeddings1))
    norm = np.linalg.norm(embeddings0) * np.linalg.norm(embeddings1)
    # shaving
    similarity = np.clip(dot / norm, -1., 1.)
    dist = np.arccos(similarity) / math.pi
    return dist

dist = distance_(embedding1, embedding2)

print(f"embedding1: {embedding1}")
print(f"embedding2: {embedding2}")
print(f"Distance: {dist}")
print(f"similarity: {1 - dist}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-14-fbaa75b82b4e>:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("/content/drive/My Drive/faceRecognition/magface_iresnet50_MS1MV2_d

embedding1: [ 6.3804255  19.14647     0.3123509  ...  0.17544618  0.
  2.710642  ]
embedding2: [ 6.78811    20.194508    0.30264223 ...  0.22445828  0.
  3.00511   ]
Distance: 0.0049760705602348715
similarity: 0.9950239294397651


In [15]:
!pip install onnx

def convert_to_onnx(model, onnx_path, img_input):
    torch.onnx.export(
        model,
        img_input,
        onnx_path,
        export_params=True,
        opset_version=11,
        do_constant_folding=True,
        input_names=["input"],
        output_names=["output"],
        dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}}
    )
    print(f"Model successfully converted to {onnx_path}")

onnx_path = "magface_iresnet50.onnx"

convert_to_onnx(model, onnx_path, preprocess_image(image1_path))

Model successfully converted to magface_iresnet50.onnx


In [16]:
!pip install onnxruntime
import onnxruntime as ort

In [17]:
def extract_embedding(onnx_path, image_path):
    image_tensor = preprocess_image(image_path).numpy()
    session = ort.InferenceSession(onnx_path)
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name
    embedding = session.run([output_name], {input_name: image_tensor})[0]
    return embedding

embedding1 = extract_embedding(onnx_path, image1_path)
embedding2 = extract_embedding(onnx_path, image2_path)

dist = distance_(embedding1.flatten(), embedding2.flatten())

print(f"embedding1: {embedding1}")
print(f"embedding2: {embedding2}")
print(f"Distance: {dist}")
print(f"similarity: {1 - dist}")

embedding1: [[ 6.380427   19.146467    0.31235018 ...  0.1754458   0.
   2.7106433 ]]
embedding2: [[ 6.788112   20.194506    0.3026447  ...  0.22445679  0.
   3.0051086 ]]
Distance: 0.0049760705602348715
similarity: 0.9950239294397651
